In [1]:
import pandas as pd
import os 

In [148]:
pd.set_option('display.max_columns', None)

In [172]:
df = pd.read_csv('../data/race_data/race_data.csv')

In [173]:
df[df['Horse_id'] == 'AUS']

,RaceIndex,Pla.,Date,RC/Track/Course,Dist.,G,RaceClass,Dr.,Rtg.,Trainer,Jockey,LBW,Win Odds,Act.Wt.,RunningPosition,Finish Time,Declar.Horse Wt.,Gear,Horse_id,Origin / Age,Colour / Sex,Import type,Sire,Dam,Dam sire
41,675,WV,21/05/2023,"ST / Turf / ""C+3""",1600,G,5,--,38,C W Chang,E C W Wong,--,--,123,--,--,--,E316,AUS,Bay / Gelding,PP,Holy Roman Emperor,Sunday Lady,Zabeel,NaN
78,748,WV,12/06/2024,"HV / Turf / ""B""",1200,G,3,--,75,C Fownes,L Ferraris,--,--,131,--,--,--,G246,AUS,Bay / Gelding,PPG,Hinchinbrook,Ennerdale,Singspiel,NaN
152,9,WV,10/09/2023,"ST / Turf / ""A""",1200,GY,3,--,62,D A Hayes,M F Poon,--,--,118,--,--,--,H263,AUS,Bay / Gelding,PP,Deep Field,Ki Ki,Kheleyf,NaN
915,182,WV,17/11/2024,"ST / Turf / ""B+2""",1200,G,4,--,50,J Size,K Teetan,--,--,129,--,--,--,J120,AUS,Chestnut / Gelding,PPG,Shooting To Win,Jaylo,Helmet,NaN
924,451,WV,25/02/2024,"ST / Turf / ""A+3""",1400,G,4,--,55,J Size,Z Purton,--,--,131,--,--,--,J120,AUS,Chestnut / Gelding,PPG,Shooting To Win,Jaylo,Helmet,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32445,804,WV,06/07/2022,"HV / Turf / ""A""",1650,G,4,--,43,P O'Sullivan,R Maia,--,--,118,--,--,--,G193,AUS,Brown / Gelding,PP,Pluck,Moporena,Arena,NaN
32556,113,WV-A,20/10/2024,"ST / Turf / ""B+2""",1400,GF,4,--,52,K H Ting,L Ferraris,--,--,128,--,--,--,J349,AUS,Bay / Gelding,PPG,Dubious,Faction,Encosta de Lago,NaN
32928,725,WV,07/06/2023,"HV / Turf / ""A""",1200,G,4,--,53,C S Shum,H Bowman,--,--,128,--,--,--,G249,AUS,Bay / Gelding,PPG,Shooting To Win,Raitza,Reset,NaN
33429,395,WV,03/02/2022,"ST / Turf / ""B+2""",1800,G,2,--,89,C Fownes,J Moreira,--,--,129,--,--,--,G103,AUS,Grey / Gelding,PP,Toronado,Bahamas,Teofilo,NaN


In [60]:
df1 = df.copy()

In [92]:
mask = df1['RC/Track/Course'].apply(lambda x: len(str(x).split('/')) != 3)

In [94]:
row = df1[mask]

In [96]:
row['RC/Track/Course'].unique()

array(['ST / AWT'], dtype=object)

# Utility function

In [61]:
# convert to new race index format
def convert_year(date_str):
    day, month, year = date_str.split('/')

    if len(year) == 4:
        year = year[2:]
    
    return f'{day}/{month}/{year}'

In [69]:
# combine race index with date to form new index
def combine_index_date(df, col1, col2, new_col):

    df['race_index'] = df[col1].astype(str) + df[col2].str.replace('/', '').astype(str)

    df = df.drop(columns = ['RaceIndex', 'Date'])

    return df


In [147]:
# clean RC, Track, Course
def clean_rc_track_course(text):
    parts = str(text).split('/')

    if len(parts) < 3:
        rc = parts[0].strip()
        track = parts[1].strip()
        course = None
    
    else:
        rc, track, course = parts[0].strip(), parts[1].strip(), parts[2].strip().replace('"', '')

    return rc, track, course

In [139]:
# clean origin, age
def clean_origin_age(text):
    parts = str(text).split('/')

    if len(parts) < 2:
        origin = parts[0].strip()
        age = None
    
    else:
        origin = parts[0].strip()
        age = parts[1].strip()
    
    return origin, age

In [154]:
# clean colour, sex
def clean_colour_sex(text):
    parts = str(text).split('/')

    colour = parts[0].strip()
    sex = parts[-1].strip()

    return colour, sex

# experiemnt

In [50]:
df1['Date'] = df1['Date'].apply(convert_year)

In [36]:
df1

,RaceIndex,Pla.,Date,RC/Track/Course,Dist.,G,RaceClass,Dr.,Rtg.,Trainer,...,Finish Time,Declar.Horse Wt.,Gear,Horse_id,Origin / Age,Colour / Sex,Import type,Sire,Dam,Dam sire
0,142,11,30/10/24,"HV / Turf / ""C+3""",1000,G,5,9,37,K L Man,...,0.58.59,1019,H-/PC-/B1/TT,J006,AUS,Chestnut / Gelding,PPG,Capitalist,Dorodansa,Bellamy Road
1,48,12,25/09/24,"HV / Turf / ""C""",1200,G,5,2,39,K L Man,...,1.11.95,1023,CP-/H1/PC1/TT,J006,AUS,Chestnut / Gelding,PPG,Capitalist,Dorodansa,Bellamy Road
2,661,10,11/05/24,"ST / Turf / ""C""",1200,GF,4,6,44,T P Yung,...,1.11.59,1013,CP1/TT,J006,AUS,Chestnut / Gelding,PPG,Capitalist,Dorodansa,Bellamy Road
3,468,14,03/03/24,"ST / Turf / ""B+2""",1600,G,4,12,47,T P Yung,...,1.37.04,1029,TT,J006,AUS,Chestnut / Gelding,PPG,Capitalist,Dorodansa,Bellamy Road
4,403,12,07/02/24,"HV / Turf / ""B""",1200,G,4,2,50,T P Yung,...,1.13.75,1019,TT1,J006,AUS,Chestnut / Gelding,PPG,Capitalist,Dorodansa,Bellamy Road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33690,278,03,23/12/23,"ST / Turf / ""C""",1400,G,3,11,68,C Fownes,...,1.22.88,1152,--,H459,AUS / 6,Bay / Gelding,PP,Impending,Isola Blu,Blackfriars
33691,240,02,10/12/23,"ST / Turf / ""A""",1200,G,3,10,68,C Fownes,...,1.09.52,1153,--,H459,AUS / 6,Bay / Gelding,PP,Impending,Isola Blu,Blackfriars
33692,185,02,19/11/23,"ST / Turf / ""B+2""",1200,GF,3,6,68,C Fownes,...,1.08.94,1155,--,H459,AUS / 6,Bay / Gelding,PP,Impending,Isola Blu,Blackfriars
33693,801,06,01/07/25,"ST / Turf / ""C""",1200,G,4,11,53,M Newnham,...,1.10.15,1127,B1,K334,AUS / 4,Grey / Gelding,PPG,Street Boss,Varanasi,Encosta de Lago


# Main function

In [163]:
def engineering(df):

    # clean format of date
    df['Date'] = df['Date'].apply(convert_year)

    # filter dates only after 10/09/2023 
    df['date_dt'] = pd.to_datetime(df['Date'], format = '%d/%m/%y')
    df = df[df['date_dt'] > pd.to_datetime('10/09/23', format = '%d/%m/%y')]
    df = df.drop(columns = ['date_dt'])

    # combine the columns to new index
    df = combine_index_date(df, 'RaceIndex', 'Date', 'race_index')

    # clean rc, track, course
    df[['rc', 'track', 'course']] = df['RC/Track/Course'].apply(clean_rc_track_course).apply(pd.Series)
    df = df.drop(columns = ['RC/Track/Course'])

    # clean origin, age
    df[['origin', 'age']] = df['Origin / Age'].apply(clean_origin_age).apply(pd.Series)
    df = df.drop(columns = ['Origin / Age'])

    # clean colour, sex
    df[['colour', 'sex']] = df['Colour / Sex'].apply(clean_colour_sex).apply(pd.Series)
    df = df.drop(columns = ['Colour / Sex'])

    # drop columns that will not be able to obtained prior matches
    df = df.drop(columns = ['LBW', 'RunningPosition', 'Finish Time'])

    # rename columns
    df = df.rename(columns = {'Dr.' : 'gate_position', 'G' : 'track_condition'})

    return df

In [164]:
df = engineering(df)

In [168]:
df[df['Horse_id'] == 'AUS']

,Pla.,Dist.,track_condition,RaceClass,gate_position,Rtg.,Trainer,Jockey,Win Odds,Act.Wt.,Declar.Horse Wt.,Gear,Horse_id,Import type,Sire,Dam,Dam sire,race_index,rc,track,course,origin,age,colour,sex
78,WV,1200,G,3,--,75,C Fownes,L Ferraris,--,131,--,G246,AUS,Hinchinbrook,Ennerdale,Singspiel,NaN,748120624,HV,Turf,B,Bay,Gelding,PPG,PPG
915,WV,1200,G,4,--,50,J Size,K Teetan,--,129,--,J120,AUS,Shooting To Win,Jaylo,Helmet,NaN,182171124,ST,Turf,B+2,Chestnut,Gelding,PPG,PPG
924,WV,1400,G,4,--,55,J Size,Z Purton,--,131,--,J120,AUS,Shooting To Win,Jaylo,Helmet,NaN,451250224,ST,Turf,A+3,Chestnut,Gelding,PPG,PPG
1677,WX,1800,G,4,--,52,K H Ting,L Ferraris,--,127,--,E176,AUS,Dalakhani,Amalgamate,Zenno Rob Roy,NaN,344170124,HV,Turf,C,Chestnut,Gelding,PPG,PPG
1763,WV-A,1000,G,4,--,60,D A Hayes,Z Purton,--,135,--,D387,AUS,Fastnet Rock,Scandinavia,Snippets,NaN,336130124,ST,Turf,C+3,Bay,Gelding,PP,PP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31021,WV-A,1200,G,3,--,66,D A Hayes,Z Purton,--,121,--,H007,AUS,Scissor Kick,Phaedra,Hussonet,NaN,18130923,HV,Turf,A,Brown,Gelding,PPG,PPG
31186,WX-A,1000,G,2,--,81,A S Cruz,Y L Chung,--,110,--,D476,AUS,Gingerbread Man,Single Spice,Not A Single Doubt,NaN,257171223,ST,Turf,B,Bay,Gelding,PP,PP
31776,WV,1400,GF,4,--,48,P C Ng,M Chadwick,--,123,--,J221,AUS,Justify,Stopshoppingmaria,More Than Ready,NaN,25150924,ST,Turf,A+3,Bay,Gelding,PPG,PPG
31994,WV,1650,G,4,--,48,J Size,NaN,--,125,--,J149,AUS,Highland Reel,Walka Country Mile,Tale Of The Cat,NaN,160061124,HV,Turf,A,Bay,Gelding,PPG,PPG


In [159]:
df['sex'].unique()

array(['Gelding', 'PPG', 'PP', 'Horse', 'Colt', 'ISG', 'Filly', 'Mare',
       'VIS'], dtype=object)

In [165]:
df[df['sex'] == 'PPG'].sort_values('Pla.')

,Pla.,Dist.,track_condition,RaceClass,gate_position,Rtg.,Trainer,Jockey,Win Odds,Act.Wt.,Declar.Horse Wt.,Gear,Horse_id,Import type,Sire,Dam,Dam sire,race_index,rc,track,course,origin,age,colour,sex
78,WV,1200,G,3,--,75,C Fownes,L Ferraris,--,131,--,G246,AUS,Hinchinbrook,Ennerdale,Singspiel,NaN,748120624,HV,Turf,B,Bay,Gelding,PPG,PPG
18787,WV,1000,GF,4,--,53,P C Ng,A Atzeni,--,130,--,G400,AUS / 6,Capitalist,Labdien,Stryker,NaN,692210525,HV,Turf,C,Bay,Gelding,PPG,PPG
19455,WV,1650,G,4,--,60,K L Man,Z Purton,--,135,--,G028,AUS,Dissident,Ruby La Rouge,Beautiful Crown,NaN,743120624,HV,Turf,B,Bay,Gelding,PPG,PPG
19610,WV,1400,G,4,--,48,A S Cruz,Y L Chung,--,120,--,J123,FR / 4,Starspangledbanner,Marechale,Anabaa,NaN,721310525,ST,Turf,B,Bay,Gelding,PPG,PPG
19657,WV,1200,GY,4,--,47,M Newnham,K C Leung,--,122,--,H197,NZ / 6,Charm Spirit,Nomisma,Thorn Park,NaN,96181023,HV,Turf,B,Bay,Gelding,PPG,PPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506,WX-A,1400,G,4,--,59,D A Hayes,H Bowman,--,135,--,J219,AUS / 5,Fastnet Rock,My Sabeel,Savabeel,NaN,699260524,ST,Turf,A,Bay,Gelding,PPG,PPG
11812,WX-A,1400,G,3,--,84,D J Whyte,H Bowman,--,134,--,H078,AUS,Capitalist,Grenada,Dolphin Street,NaN,703260524,ST,Turf,A,Bay,Gelding,PPG,PPG
26921,WX-A,1000,G,5,--,28,A S Cruz,Y L Chung,--,120,--,H371,AUS,Shalaa,Nataya Rose,Canny Lad,NaN,251111224,HV,Turf,B,Bay,Gelding,PPG,PPG
32316,WX-A,1000,G,5,--,34,C Fownes,A Badel,--,129,--,H046,NZ,Swiss Ace,Repulse Bay,Darci Brahma,NaN,251111224,HV,Turf,B,Bay,Gelding,PPG,PPG


In [138]:
df.columns

Index(['Pla.', 'Dist.', 'track_condition', 'RaceClass', 'gate_position',
       'Rtg.', 'Trainer', 'Jockey', 'Win Odds', 'Act.Wt.', 'Declar.Horse Wt.',
       'Gear', 'Horse_id', 'Origin / Age', 'Colour / Sex', 'Import type',
       'Sire', 'Dam', 'Dam sire', 'race_index', 'rc', 'track', 'course'],
      dtype='object')